# Calculate cells movement

### calculating 3D cell movements and saving in csv file.

### 3次元的なROIの移動距離を算出するコードです。クマムシ用 Choose Stardist env

In [4]:
#重心計算の関数
from __future__ import annotations  # ← 必ず一番最初（コメントやdocstringの後）

import os
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
import numpy as np



import re
from glob import glob
from pathlib import Path

import numpy as np

from scipy import ndimage


from CellTracker.stardistwrapper import load_2d_slices_at_time




def calculate_center(tracked_labels_path :str, raw_path: str | dict):
    tracked_labels_path=Path(tracked_labels_path)
    filenames = glob(str(tracked_labels_path / "*t*.tif"))
    assert len(filenames) > 0, f"No labels files were found in {tracked_labels_path / '*t*.tif'}"
    numbers = [int(re.findall(r"t(\d+)", f)[0]) for f in filenames]
    smallest_number = min(numbers)
    largest_number = max(numbers)
    for t in range(smallest_number, largest_number + 1):
        print(f"t={t}...", end="\r")
        try:
            # Load 2D slices at time t
            raw = load_2d_slices_at_time(raw_path, t=t)
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Raw images at t={t - 1} cannot be loaded! Stop calculation!")
            break

        try:
            # Load 2D slices at time t
            labels_img = load_2d_slices_at_time(str(tracked_labels_path / "*t%06i*.tif"), t=t, do_normalize=False)
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Label images at t={t - 1} cannot be loaded!")
            if t == smallest_number:
                print("Warning: stop calculation!")
                break
            else:
                print(f"Warning: skip volume {t - 1}!")
                cell_position[t - smallest_number, :] = np.nan
                continue

        if t == smallest_number:
            cell_num = np.max(labels_img)
            label_number = list(range(cell_num+1))
            cell_position = np.zeros((largest_number - smallest_number + 1,cell_num, 3))


    


        center_of_labels = ndimage.center_of_mass(raw, labels_img,label_number[1:cell_num+1])
        cell_position[t-smallest_number,:] = center_of_labels[:]
        


    return cell_position

ModuleNotFoundError: No module named 'CellTracker'

In [ ]:
#生データとラベルデータを入力し、重心計算実行
raw_path = {"h5_file":r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=566-665\raw_t=566~665.h5",
            "dset": "default",
            "channel": 0}


tracked_labels_path =r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=566-665\merged_5d_566-665"

#raw_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_reconst\*t%04d*.tif"


center_of_labels = calculate_center(tracked_labels_path = tracked_labels_path, raw_path=raw_path)



In [ ]:
#上で重心計算した結果をセーブする

np.save(r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665\center_of_labels_t=566~665.npy" ,center_of_labels )
# center_of_labels=np.load("center_of_labels_t=201-221.npy")
print(center_of_labels.shape)

In [ ]:
#下の計算をするための準備。ここで重心座標をロード
#center_of_labels=np.load(r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=226~364\center_of_labels_t=226~364.npy")
center_of_labels=np.load(r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\labels_raw_t=1-225\center_of_labels_t001-225.npy")


In [ ]:
#ROI1-ROI2の距離を計算

from scipy.spatial import distance

csv_filename = r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665\tardigrade_t=566~665_L4leg_labels_distance_1-2.csv"

#You can change file name above.

#cell_movement = center_of_labels
#cell_movement = np.zeros((center_of_labels.shape[0],center_of_labels.shape[1]-1))
cell_movement =np.zeros((center_of_labels.shape[0],4))
print(center_of_labels.shape)
label2_1_vector=np.zeros([center_of_labels.shape[0],3])
label2_1_vector[:,0]=center_of_labels[:,1,0] - center_of_labels[:,0,0]
label2_1_vector[:,1]=center_of_labels[:,1,1] - center_of_labels[:,0,1]
label2_1_vector[:,2]=center_of_labels[:,1,2] - center_of_labels[:,0,2]

print(label2_1_vector[1])
for t in range(center_of_labels.shape[0]):

    cell_position_t1 = center_of_labels[t,0]
    cell_position_t2 = center_of_labels[t,1]
    print(cell_position_t2)
    print(cell_position_t1)
    zx_center_of_labels_t1 = np.stack([cell_position_t1[0],cell_position_t1[2]],0)
    zx_center_of_labels_t2 = np.stack([cell_position_t2[0],cell_position_t2[2]],0)

    three_d_dist = distance.euclidean(cell_position_t1, cell_position_t2)
    #xy_d_dist = distance.euclidean(cell_position_t1[1:2], cell_position_t2[1:2])
    #xz_d_dist = distance.euclidean(zx_center_of_labels_t1, zx_center_of_labels_t2)
    #yz_d_dist = distance.euclidean(cell_position_t1[0:1], cell_position_t2[0:1])

    #xy_d_dist = distance.euclidean(cell_position_t1[:2],  cell_position_t2[:2])
    #yz_d_dist = distance.euclidean(cell_position_t1[1:],  cell_position_t2[1:])

    cell_movement[t, 0] = three_d_dist*0.4095
    cell_movement[t, 1] = xy_d_dist*0.4095
    cell_movement[t, 2] = xz_d_dist*0.4095
    cell_movement[t, 3] = yz_d_dist*0.4095



csv_filename = os.path.join(tracked_labels_path, csv_filename)
timings = np.arange(1, cell_movement.shape[0]+1).reshape(-1, 1)
movement_with_timings = np.concatenate((timings, cell_movement), axis=1)
headers = ['timing'] + ['cell' + str(i) for i in range(1, cell_movement.shape[1])]
np.savetxt(csv_filename, movement_with_timings, delimiter=',', fmt='%.3f', header=','.join(headers), comments='')
print(f"The cell movements have been saved in {csv_filename}")